Импортируем библиотеки

In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
from tqdm import tqdm
from transformers import ViTImageProcessor, ViTModel
from torch.nn import TransformerEncoder, TransformerEncoderLayer


In [2]:

seed = 732
print(f'seed: {seed}')

random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

seed: 732


In [3]:
data = pd.read_csv('/kaggle/input/preprocessed-tabular/preprocessed_data.csv')
X_tab = data.drop(columns=['Цена'])
y = data['Цена']

In [ ]:
X_tab

In [4]:
class HouseDataset(Dataset):
    def __init__(self, X_tab, pictures_dir):
        self.X_tab = X_tab  # Табличные данные
        self.pictures_dir = pictures_dir  # Директория с изображениями
        

    def __len__(self):
        return len(self.X_tab)

    def __getitem__(self, idx):
        house_idx = int(self.X_tab.iloc[idx, 0])  # Обращение к элементу по индексу и столбцу

        # Чтение и обработка изображений для этого дома
        img_dir = os.path.join(self.pictures_dir, f'ann_{str(house_idx)}')
        img_names = os.listdir(img_dir)
        images = []
        for img_name in img_names:
            img_path = os.path.join(img_dir, img_name)
            image = Image.open(img_path).convert('RGB')
            images.append(image)

        return house_idx, images


In [5]:
dataset = HouseDataset(X_tab, '/kaggle/input/qwesdfsa/train/train')



In [15]:
image_proccesor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
extractor = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k').to('cuda')

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [ ]:


embed = {'id':[],'embeddings':[]}

for house_idx, images in tqdm(dataset):
    house_idx = house_idx
    inputs = image_proccesor(images=images, return_tensors="pt").to('cuda')
    
    with torch.no_grad():
        outputs = extractor(**inputs)
        last_hidden_states = outputs.last_hidden_state

        image_embedding_mean = last_hidden_states.mean(dim=1)# Среднее по изображениям для каждого дома
        mean_emb =image_embedding_mean = image_embedding_mean.mean(dim=0)# Среднее по изображениям для каждого дома

        embed['id'].append(house_idx)
        embed['embeddings'].append(mean_emb.to('cpu').numpy())
    
df_embed = pd.DataFrame(embed)
df_data = X_tab
merged_df = pd.merge(df_embed, df_data, on='id')

In [9]:
merged_df.to_csv('data_for_training_with_tensors',index=False)

In [76]:
embeddings_for_train = merged_df['embeddings']
embeddings_for_train.to_csv('embeddings_for_train.csv',index=False)


In [4]:
import optuna
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error


In [11]:

# Загрузка датасета
data = pd.read_csv('/kaggle/input/ockboost/data_for_training_with_tensors.csv')
y = pd.read_csv('/kaggle/input/preprocessed-tabular/preprocessed_data.csv')['Цена']
data['embeddings'] = data['embeddings'].apply(lambda row: [x for x in row.replace('\n', '').replace('[','').replace(']','').split(' ')]).apply(lambda row:[float(x) for x in row if x not in ('')])
X_train, X_valid, y_train, y_valid = train_test_split(data, y, test_size=0.2, random_state=732)


In [ ]:


# Функция оптимизации
def objective(trial):
    # Определяем гиперпараметры для оптимизации
    params = {
        "iterations": trial.suggest_int("iterations", 100, 10000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "depth": trial.suggest_int("depth", 4, 16),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 10),
        "random_strength": trial.suggest_float("random_strength", 1e-3, 10),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0, 1),
        "border_count": trial.suggest_int("border_count", 1, 255),
        "loss_function": "RMSE",
        "embedding_features":[1],
        "cat_features":categorical_columns    }

    # Обучаем модель
    model = CatBoostRegressor(**params,)
    model.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=250)

    # Предсказываем и оцениваем качество
    preds = model.predict(X_valid)
    mape = mean_absolute_percentage_error(y_valid, preds)
    
    # Возвращаем метрику для минимизации
    return mape

# Создаем исследование Optuna и запускаем оптимизацию
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

# Лучшие параметры
print("Best trial:")
print(study.best_trial.params)

[I 2024-09-09 19:08:22,501] A new study created in memory with name: no-name-6726333a-c283-4112-aa5f-17209f2ac6b8


0:	learn: 305896.0597678	test: 309698.1839863	best: 309698.1839863 (0)	total: 15.6ms	remaining: 6.12s
250:	learn: 173618.8376155	test: 193450.0937767	best: 193450.0937767 (250)	total: 1m 35s	remaining: 53.6s
391:	learn: 136772.0418664	test: 176738.4892668	best: 176738.4892668 (391)	total: 2m 48s	remaining: 0us

bestTest = 176738.4893
bestIteration = 391



[I 2024-09-09 19:13:07,384] Trial 0 finished with value: 0.4564038520903767 and parameters: {'iterations': 392, 'learning_rate': 0.03405811835041115, 'depth': 14, 'l2_leaf_reg': 5.283872927396469, 'random_strength': 5.895035780805282, 'bagging_temperature': 0.8320143959782885, 'border_count': 251}. Best is trial 0 with value: 0.4564038520903767.


0:	learn: 304897.1410042	test: 308700.1303821	best: 308700.1303821 (0)	total: 13.6ms	remaining: 1m 33s
250:	learn: 183727.6191223	test: 189250.0420109	best: 189250.0420109 (250)	total: 3.28s	remaining: 1m 26s
500:	learn: 164812.9953463	test: 174621.1713128	best: 174621.1713128 (500)	total: 6.54s	remaining: 1m 23s
750:	learn: 156882.6649775	test: 170172.5728888	best: 170172.2606231 (749)	total: 9.8s	remaining: 1m 20s
1000:	learn: 151637.8116532	test: 167489.3823132	best: 167477.3593732 (999)	total: 13.1s	remaining: 1m 17s
1250:	learn: 147218.8305178	test: 165447.1367556	best: 165447.0106217 (1247)	total: 16.9s	remaining: 1m 16s
1500:	learn: 143580.2533880	test: 163925.2507667	best: 163915.3887421 (1499)	total: 20.3s	remaining: 1m 12s
1750:	learn: 140512.1706796	test: 162789.5400501	best: 162789.5400501 (1750)	total: 23.6s	remaining: 1m 9s
2000:	learn: 137720.3114286	test: 161733.2523131	best: 161727.7572315 (1988)	total: 26.9s	remaining: 1m 5s
2250:	learn: 135089.0890677	test: 160689.14

[I 2024-09-09 19:16:37,027] Trial 1 finished with value: 0.378332099972231 and parameters: {'iterations': 6893, 'learning_rate': 0.04222673880214367, 'depth': 5, 'l2_leaf_reg': 7.687322071481996, 'random_strength': 2.2844356308690954, 'bagging_temperature': 0.671607508521011, 'border_count': 74}. Best is trial 1 with value: 0.378332099972231.


...

0:	learn: 301316.2102954	test: 305530.7532958	best: 305530.7532958 (0)	total: 82.5ms	remaining: 7m 29s
250:	learn: 134164.2642732	test: 168927.6327513	best: 168927.6327513 (250)	total: 23.1s	remaining: 7m 58s
500:	learn: 110707.4467044	test: 163163.6453580	best: 163163.6453580 (500)	total: 46.6s	remaining: 7m 40s
750:	learn: 97530.7089267	test: 161220.7947258	best: 161220.6561195 (749)	total: 1m 10s	remaining: 7m 19s
1000:	learn: 85631.3285684	test: 159915.1712120	best: 159905.0057460 (996)	total: 1m 34s	remaining: 6m 58s
1250:	learn: 76823.7849155	test: 158893.4945554	best: 158893.4945554 (1250)	total: 1m 57s	remaining: 6m 35s
1500:	learn: 68006.3472396	test: 158189.4811101	best: 158189.4811101 (1500)	total: 2m 21s	remaining: 6m 11s
1750:	learn: 60653.0686378	test: 157605.5622129	best: 157605.5622129 (1750)	total: 2m 45s	remaining: 5m 49s
2000:	learn: 53404.7676295	test: 157139.1653261	best: 157139.1653261 (2000)	total: 3m 9s	remaining: 5m 26s
2250:	learn: 47660.7166915	test: 156912.3

[I 2024-09-09 19:48:57,523] Trial 5 finished with value: 0.3784747607657604 and parameters: {'iterations': 5454, 'learning_rate': 0.045100015684854756, 'depth': 12, 'l2_leaf_reg': 8.011789877647256, 'random_strength': 0.6923350604819036, 'bagging_temperature': 0.9835825857038814, 'border_count': 43}. Best is trial 4 with value: 0.36619029638983114.


0:	learn: 306973.1639993	test: 310708.4455926	best: 310708.4455926 (0)	total: 4.52ms	remaining: 32.2s
250:	learn: 144123.0309863	test: 181234.8307651	best: 181234.8307651 (250)	total: 1m 47s	remaining: 49m 39s
500:	learn: 77660.0763943	test: 168672.7789098	best: 168672.7789098 (500)	total: 4m 26s	remaining: 58m 54s
750:	learn: 50062.6711910	test: 166537.5240054	best: 166537.5240054 (750)	total: 7m 5s	remaining: 1h 18s
1000:	learn: 34518.0645946	test: 165825.3977431	best: 165824.7116806 (997)	total: 9m 43s	remaining: 59m 31s
1250:	learn: 25391.9108598	test: 165386.3049141	best: 165386.3049141 (1250)	total: 12m 21s	remaining: 58m 5s
1500:	learn: 19147.6063472	test: 165107.6804433	best: 165105.2503097 (1488)	total: 15m 1s	remaining: 56m 18s
1750:	learn: 14393.8026991	test: 164980.8888937	best: 164980.8888937 (1750)	total: 17m 40s	remaining: 54m 15s
2000:	learn: 11038.3511925	test: 164906.7134904	best: 164900.8449526 (1980)	total: 20m 19s	remaining: 52m 1s
2250:	learn: 8770.3399905	test: 1

[I 2024-09-09 21:05:51,365] Trial 6 finished with value: 0.40690268669065893 and parameters: {'iterations': 7115, 'learning_rate': 0.049475533232893734, 'depth': 15, 'l2_leaf_reg': 5.855137649303994, 'random_strength': 6.816088795300755, 'bagging_temperature': 0.42954127198956005, 'border_count': 79}. Best is trial 4 with value: 0.36619029638983114.


0:	learn: 300101.9959499	test: 305036.0457682	best: 305036.0457682 (0)	total: 1.29s	remaining: 3h 4m 5s
250:	learn: 69591.7549733	test: 173302.1150085	best: 173302.1150085 (250)	total: 5m 1s	remaining: 2h 46m 45s
500:	learn: 26957.7783871	test: 170980.2905296	best: 170973.6169188 (499)	total: 10m 31s	remaining: 2h 49m 26s
750:	learn: 13237.3214062	test: 170497.6873213	best: 170497.3224399 (740)	total: 16m 3s	remaining: 2h 47m 9s
1000:	learn: 7325.2880901	test: 170358.0054853	best: 170354.4007490 (976)	total: 21m 31s	remaining: 2h 42m 47s
1250:	learn: 4317.8778241	test: 170343.3770162	best: 170341.3496635 (1217)	total: 27m	remaining: 2h 38m 4s
1500:	learn: 2810.4276753	test: 170330.6494004	best: 170330.6494004 (1500)	total: 32m 33s	remaining: 2h 33m 20s
1750:	learn: 1851.7365991	test: 170316.6128467	best: 170316.6128467 (1750)	total: 38m 6s	remaining: 2h 28m 26s
2000:	learn: 1178.4232315	test: 170309.9946970	best: 170309.9946970 (2000)	total: 43m 37s	remaining: 2h 23m 13s
2250:	learn: 8

In [13]:
df_test = pd.read_csv('/kaggle/input/qwesdfsa/public_test.csv')

In [ ]:
df_train_no_label_encoded  = pd.read_csv('/kaggle/input/qwesdfsa/train.csv')

In [57]:
def fill_null_test(feature, method):
    if method == 'mean':
        df_test[feature] = df_test[feature].fillna(df_train_no_label_encoded[feature].mean())
        return df_test[feature].isnull().sum()
    elif method == 'mode':
        df_test[feature] = df_test[feature].fillna(df_train_no_label_encoded[feature].mode()[0])
        return df_test[feature].isnull().sum()
    else:
        return 'Method Error: Choose mean or mode in second input.'
mean = 'mean'
mode = 'mode'
fill_null_test('Площадь',mean)
fill_null_test('Этаж',mode)
fill_null_test('Размер_участка',mean)
fill_null_test('Расход_тепла',mean)
fill_null_test('Ктгр_энергоэффективности',mode)
fill_null_test('Кво_вредных_выбросов',mean)
fill_null_test('Ктгр_вредных_выбросов',mode)
fill_null_test('Направление',mode)
fill_null_test('Кво_спален',mode)
fill_null_test('Кво_комнат',mode)
fill_null_test('Кво_фото',mode)


0

In [59]:
test_dataset = HouseDataset(df_test, '/kaggle/input/testphoto/public_test')

In [14]:
categorical_columns = df_test.select_dtypes(include=['object']).columns.tolist()

In [15]:
categorical_columns

['Тип_жилья',
 'Город',
 'Ктгр_энергоэффективности',
 'Ктгр_вредных_выбросов',
 'Направление']

In [65]:
from sklearn.preprocessing import LabelEncoder
# Создаем объект LabelEncoder
label_encoders = {}

# Определим, какие столбцы являются категориальным

# Применим LabelEncoder к каждому категориальному столбцу
for col in categorical_columns:
    le = LabelEncoder()
    df_test[col] = le.fit_transform(df_test[col])

In [16]:
embed = {'id':[],'embeddings':[]}

for house_idx, images in tqdm(test_dataset):
    house_idx = house_idx
    inputs = image_proccesor(images=images, return_tensors="pt").to('cuda')
    
    with torch.no_grad():
        outputs = extractor(**inputs)
        last_hidden_states = outputs.last_hidden_state

        image_embedding_mean = last_hidden_states.mean(dim=1)# Среднее по изображениям для каждого дома
        mean_emb =image_embedding_mean = image_embedding_mean.mean(dim=0)# Среднее по изображениям для каждого дома

        embed['id'].append(house_idx)
        embed['embeddings'].append(mean_emb.to('cpu').numpy())
    


100%|██████████| 5605/5605 [08:17<00:00, 11.26it/s]


In [66]:
test_df_embed = pd.DataFrame(embed)
test_merged_df = pd.merge(test_df_embed, df_test, on='id')

In [73]:
test_df_embed.to_csv('embeddings_for_test.csv',index=False)

In [68]:
outputs = model.predict(test_merged_df)

In [69]:
outputs

array([750958.43419683, 137619.96069622, 161638.00021297, ...,
       214472.3586224 , 372463.73472711, 474541.49991039])

In [70]:
sub_frame = pd.DataFrame(columns = ['id','Цена'])
sub_frame['id'] = df_test['id']
sub_frame['Цена'] = outputs

In [72]:
sub_frame.to_csv('public_submission_seed_732.csv',index = False)